In [201]:
import numpy as np
import xarray as xr
import seaborn as sns

from pathlib import Path
from time import time
from tqdm.autonotebook import tqdm
from sklearn.preprocessing import StandardScaler             # Feature scaling
from sklearn.model_selection import train_test_split         # Split data into train and test set
from sklearn.metrics import classification_report            # Summary of classifier performance
from sklearn.metrics import confusion_matrix                 # Confusion matrix
from sklearn.metrics import accuracy_score

from utils import get_df

# Automatically prints execution time for the individual cells
%load_ext autotime

# Automatically reloads functions defined in external files
%load_ext autoreload
%autoreload 2

# Set xarray to use html as display_style
xr.set_options(display_style="html")

# Tell matplotlib to plot directly in the notebook
%matplotlib inline  

# The path to the project (so absoute file paths can be used throughout the notebook)
PROJ_PATH = Path.cwd().parent

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 264 ms


In [6]:
netcdf_path = (PROJ_PATH / 'data' / 'processed' / 'FieldPolygons2019_stats').with_suffix('.nc')
ds = xr.open_dataset(netcdf_path, engine="h5netcdf")
ds  # Remember to close the dataset before the netcdf file can be rewritten in cells above

<xarray.Dataset>
Dimensions:         (date: 77, field_id: 67312, polarization: 3)
Coordinates:
  * date            (date) datetime64[ns] 2018-07-08 2018-07-14 ... 2019-10-31
  * field_id        (field_id) int32 61853445 61952339 ... 62131272 63199619
  * polarization    (polarization) object 'VH' 'VV' 'VV-VH'
Data variables:
    afgkode         (field_id) int16 ...
    afgroede        (field_id) object ...
    cvr             (field_id) object ...
    gb              (field_id) float32 ...
    gbanmeldt       (field_id) float32 ...
    imk_areal       (field_id) float32 ...
    journalnr       (field_id) object ...
    marknr          (field_id) object ...
    pass_mode       (date) object ...
    relative_orbit  (date) int16 ...
    satellite       (date) object ...
    stats_max       (field_id, date, polarization) float32 ...
    stats_mean      (field_id, date, polarization) float32 ...
    stats_median    (field_id, date, polarization) float32 ...
    stats_min       (field_id, date, polarization) float32 ...
    stats_std       (field_id, date, polarization) float32 ...

time: 209 ms


In [7]:
ds.close()

time: 24.7 ms


In [344]:
# Convert the xarray dataset to pandas dataframe
df = ds.to_dataframe()
df = df.reset_index()  # Removes MultiIndex
df = df.drop(columns=['cvr', 'gb', 'gbanmeldt', 'journalnr', 'marknr', 'pass_mode', 'relative_orbit'])
df = df.dropna()

time: 11.2 s


In [151]:
# Create the NumPy arrays to be used by scikit-learn. Start by finding number of fields, dates, and polarizations.
num_fields = len(df['field_id'].unique())
num_dates = len(df['date'].unique())
num_polarizations = len(df['polarization'].unique())

# Get the labels (and ensure that our dataframe is formatted as it is suppposed to be)
for i, date in enumerate(df['date'].unique()):  # Loop over all dates
    # Get lists with afgkode and field_ids for all fields for a single date
    df_date = df[df['date'] == date]   # Extract a df with all values for that date
    y = df_date['afgkode'].iloc[::num_polarizations].values  # Extract 'afgkode' from every N'th row 
    y_field_id = df_date['field_id'].iloc[::num_polarizations].values  # Extract 'field_id' from every N'th row
    
    # Store the lists from the first date
    if i == 0:  
        y_initial = y
        y_field_id_initial = y_field_id
    
    # Check that the lists for every date matches the first date
    assert np.array_equal(y, y_initial)
    assert np.array_equal(y_field_id, y_field_id_initial)
    
# The feature array should have all features (all polarizations for all dates) as a single row per field.
# NOTE: This is a quite inefficient implementation, but it is simple and easy to understand
print("Converting dataframe to NumPy feature array")
X = np.zeros((num_fields, num_dates*num_polarizations))  # Initialize array
for i, field_id in enumerate(tqdm(y_field_id)):  # Loop over all fields
    df_field = df[df['field_id'] == field_id]  # Extract df for the specific field
    X[i, :] = df_field['stats_mean'].values  # Extract the values ('stats_mean') and insert them into feature array
    # NOTE: Test with using the stats_std also for classification
    #X[i, ??:] = df_field['stats_std'].values

# Print numbers and shapes to give impression of dataset size
print(f"Number of fields: {num_fields}")
print(f"Number of dates: {num_dates}")
print(f"Number of polarizations: {num_polarizations}")
print(f"Shape of feature array: {np.shape(X)}")
print(f"Shape of label array: {np.shape(y)}")


Number of fields: 67312
Number of dates: 77
Number of polarizations: 3
Shape of feature array: (67312, 231)
Shape of label array: (67312,)
time: 18min 2s


In [342]:
for i, polarization in enumerate(['VV', 'VH', 'VV-VH']):
    df_polarization = get_df(polygons_year=2019, 
                             satellite_dates=slice('2018-01-01', '2018-12-31'), 
                             fields='all', 
                             satellite='all', 
                             polarization=polarization,
                             netcdf_path=netcdf_path)
    
    # Extract a mapping of field_ids to crop type
    if i == 0:
        df_sklearn = df_polarization[['field_id', 'afgkode', 'afgroede']]
    
    # Pivot the df (https://stackoverflow.com/a/37790707/12045808)
    df_polarization = df_polarization.pivot(index='field_id', columns='date', values='stats_mean')
    
    # Add polarization to column names
    df_polarization.columns = [str(col)[:10]+f'_{polarization}' for col in df_polarization.columns]  
    
    # Merge the polarization dataframes into one dataframe
    df_polarization = df_polarization.reset_index()  # Creates new indices and a 'field_id' column (field id was used as indices before)
    df_sklearn = pd.merge(df_sklearn, df_polarization, on='field_id') 
        
# Drop fields having any date with a nan value, and pick num_fields from the remainder
df_sklearn = df_sklearn.dropna()
df_sklearn = df_sklearn.drop_duplicates().reset_index(drop=True)

time: 8.22 s


In [343]:
df_sklearn

,field_id,afgkode,afgroede,2018-07-08_VV,2018-07-14_VV,2018-07-20_VV,2018-07-26_VV,2018-08-01_VV,2018-08-07_VV,2018-08-13_VV,...,2018-10-24_VV-VH,2018-10-30_VV-VH,2018-11-05_VV-VH,2018-11-11_VV-VH,2018-11-29_VV-VH,2018-12-05_VV-VH,2018-12-11_VV-VH,2018-12-17_VV-VH,2018-12-23_VV-VH,2018-12-29_VV-VH
0,61853445,151,"Kartofler, stivelses-",-12.705882,-12.891666,-15.034782,-13.612904,-11.656935,-13.682540,-14.923077,...,13.450000,11.100000,11.689922,11.190840,12.085271,12.900000,13.152174,13.727273,12.300813,12.758333
1,61952339,151,"Kartofler, stivelses-",-14.428169,-13.538243,-12.967560,-14.758087,-14.744350,-15.727401,-9.550071,...,6.966292,5.642253,5.822129,5.410184,5.469014,6.409605,6.773240,7.475989,6.326733,5.827440
2,61952436,151,"Kartofler, stivelses-",-13.846774,-10.658731,-14.063492,-16.452381,-15.198413,-16.307087,-10.436508,...,5.825397,6.377953,4.301587,4.536000,5.768000,6.428571,7.346774,7.817461,5.865079,6.322581
3,61952337,151,"Kartofler, stivelses-",-13.226611,-12.409185,-15.271579,-16.343815,-15.458159,-15.752621,-10.255814,...,7.153361,5.490405,5.919662,6.029598,6.471816,6.008351,8.846154,8.774327,6.766046,5.678723
4,61952348,151,"Kartofler, stivelses-",-14.373638,-13.537199,-15.182807,-16.214518,-15.483696,-16.474836,-10.902839,...,7.424508,5.773667,4.821156,4.890110,5.721491,6.008772,7.541712,7.651138,6.103448,5.800650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55060,62432189,252,"Permanent græs, normalt udbytte",-16.536232,-17.422535,-18.144928,-16.614286,-16.432837,-16.652174,-12.514706,...,7.405797,4.971014,5.803030,4.666667,5.275362,4.956522,5.826087,5.029851,5.147059,5.188406
55061,66346089,252,"Permanent græs, normalt udbytte",-14.166667,-14.224138,-14.346154,-13.714286,-15.160000,-14.053572,-10.236363,...,5.129630,4.298245,3.150943,5.851852,6.696429,6.160714,5.274510,6.244898,7.020408,5.980000
55062,62588991,252,"Permanent græs, normalt udbytte",-15.951456,-16.282827,-15.820000,-15.336735,-14.386139,-14.590000,-12.339806,...,7.795918,6.828283,7.894231,6.656863,6.336735,5.683673,6.792079,7.787879,8.020202,7.210000
55063,62174897,252,"Permanent græs, normalt udbytte",-12.902778,-13.881119,-13.558620,-14.078014,-14.251700,-14.443662,-9.924658,...,1.916667,4.808511,5.652778,7.090278,7.062069,6.305555,7.333333,7.448276,8.244755,7.486111


time: 63.3 ms


In [345]:
df

,date,field_id,polarization,afgkode,afgroede,imk_areal,satellite,stats_max,stats_mean,stats_median,stats_min,stats_std
0,2018-07-08,61853445,VH,151,"Kartofler, stivelses-",1.83,S1B,-17.0,-23.705883,-23.0,-31.0,3.019831
1,2018-07-08,61853445,VV,151,"Kartofler, stivelses-",1.83,S1B,-9.0,-12.705882,-12.0,-18.0,1.898597
2,2018-07-08,61853445,VV-VH,151,"Kartofler, stivelses-",1.83,S1B,17.0,10.470589,10.0,4.0,3.046071
3,2018-07-08,61952339,VH,151,"Kartofler, stivelses-",6.02,S1B,-15.0,-20.635212,-20.0,-30.0,2.294672
4,2018-07-08,61952339,VV,151,"Kartofler, stivelses-",6.02,S1B,-10.0,-14.428169,-15.0,-21.0,2.151264
...,...,...,...,...,...,...,...,...,...,...,...,...
15549064,2019-10-31,62174897,VV,252,"Permanent græs, normalt udbytte",1.88,S1B,-6.0,-10.908451,-11.0,-16.0,1.964132
15549065,2019-10-31,62174897,VV-VH,252,"Permanent græs, normalt udbytte",1.88,S1B,15.0,7.605634,7.0,2.0,2.391174
15549069,2019-10-31,63199619,VH,252,"Permanent græs, normalt udbytte",3.29,S1B,-15.0,-20.128471,-20.0,-28.0,2.366026
15549070,2019-10-31,63199619,VV,252,"Permanent græs, normalt udbytte",3.29,S1B,-7.0,-13.510417,-14.0,-19.0,2.381912


time: 46.5 ms


In [ ]:
df_sklearn[df_cancer['diagnosis'] == "B"].describe()

In [329]:
df_new = pd.merge(df_crop_types, df, on='field_id')  # The field_ids are the indices
df_new.drop_duplicates().reset_index(drop=True)

,field_id,afgkode,afgroede,2019-01-04_VV,2019-01-10_VV,2019-01-04_VH,2019-01-10_VH,2019-01-04_VV-VH,2019-01-10_VV-VH
0,61853445,151,"Kartofler, stivelses-",-10.363636,-9.380165,-24.628788,-23.933884,13.750000,14.099174
1,61952339,151,"Kartofler, stivelses-",-10.337588,-13.622535,-18.802837,-21.688732,7.960284,7.580282
2,61952436,151,"Kartofler, stivelses-",-12.128000,-14.841269,-19.912001,-23.119047,7.208000,7.706349
3,61952337,151,"Kartofler, stivelses-",-12.845511,-15.162447,-21.711901,-23.934599,8.388309,8.291140
4,61952348,151,"Kartofler, stivelses-",-13.540305,-15.340635,-22.246187,-24.354874,8.233115,8.530121
...,...,...,...,...,...,...,...,...,...
55390,62432189,252,"Permanent græs, normalt udbytte",-14.043478,-16.132353,-22.869566,-23.029411,8.275362,6.441176
55391,66346089,252,"Permanent græs, normalt udbytte",-11.188680,-13.732142,-19.528301,-22.285715,7.830189,8.035714
55392,62588991,252,"Permanent græs, normalt udbytte",-13.653465,-15.142858,-22.217821,-24.071428,8.089108,8.510204
55393,62174897,252,"Permanent græs, normalt udbytte",-8.720280,-10.253521,-17.384615,-21.035212,8.104895,10.232394


time: 96.4 ms


In [178]:
# Create a train/test split using 30% test size.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
class_names = df['afgroede'].unique()

time: 939 ms


In [179]:
def evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=False):
    """
    This function evaluates a classifier. It measures training and prediction time, and 
    prints performance metrics and a confustion matrix. The returned classifier and 
    scaler are fitted to the training data, and can be used to predict new samples.
    """
    
    # Perform feature scaling
    scaler = StandardScaler()  # Scale to mean = 0 and std_dev = 1
    if feature_scale:
        # ====================== YOUR CODE HERE =======================

        X_train = scaler.fit_transform(X_train)  # Fit to training data and then scale training data
        X_test = scaler.transform(X_test)  # Scale test data based on the scaler fitted to the training data

        # =============================================================
        
    # Store the time so we can calculate training time later
    t0 = time()

    # Fit the classifier on the training features and labels
    # ====================== YOUR CODE HERE =======================

    clf.fit(X_train, y_train)

    # =============================================================
    
    # Calculate and print training time
    print("Training time:", round(time()-t0, 4), "s")

    # Store the time so we can calculate prediction time later
    t1 = time()
    
    # Use the trained classifier to classify the test data
    # ====================== YOUR CODE HERE =======================

    predictions = clf.predict(X_test)

    # =============================================================
    
    # Calculate and print prediction time
    print("Prediction time:", round(time()-t1, 4), "s")

    # Evaluate the model
    train_accuracy = clf.score(X_train, y_train)
    test_accuracy = clf.score(X_test, y_test)
    report = classification_report(y_test, predictions, target_names=class_names)

    # Print the reports
    print("\nReport:\n")
    print("Train accuracy: {}".format(round(train_accuracy, 4)))
    print("Test accuracy: {}".format(round(test_accuracy, 4)))
    print("\n", report)
    
    # Plot confusion matrices
    cnf_matrix = confusion_matrix(y_test, predictions)
    plot_confusion_matrix(cnf_matrix, classes=class_names)
    
    # Return the trained classifier to be used on future predictions
    return clf, scaler

time: 27.8 ms


In [185]:
from sklearn.linear_model import LogisticRegression          

# Instantiate classifier.
clf = LogisticRegression(solver='newton-cg')

# Evaluate classifier without feature scaling
#clf_trained, _ = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=False)
print(np.max(X))

nan
time: 43.7 ms


In [240]:
df = get_df(polygons_year=2019, 
            satellite_dates=slice('2019-01-01', '2019-03-31'), 
            fields='all', 
            satellite='all', 
            polarization='VV',
            netcdf_path=netcdf_path)
df

,date,field_id,afgkode,afgroede,imk_areal,pass_mode,polarization,relative_orbit,satellite,stats_max,stats_mean,stats_median,stats_min,stats_std
0,2019-01-04,61853445,151,"Kartofler, stivelses-",1.83,DSC,VV,139,S1B,-7.0,-10.363636,-10.0,-16.0,2.230208
1,2019-01-04,61952339,151,"Kartofler, stivelses-",6.02,DSC,VV,139,S1B,-5.0,-10.337588,-10.0,-16.0,2.076431
2,2019-01-04,61952436,151,"Kartofler, stivelses-",1.84,DSC,VV,139,S1B,-9.0,-12.128000,-12.0,-17.0,1.658800
3,2019-01-04,61952337,151,"Kartofler, stivelses-",5.52,DSC,VV,139,S1B,-7.0,-12.845511,-13.0,-21.0,1.925853
4,2019-01-04,61952348,151,"Kartofler, stivelses-",8.34,DSC,VV,139,S1B,-8.0,-13.540305,-13.0,-21.0,1.984693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942363,2019-03-29,66346089,252,"Permanent græs, normalt udbytte",1.21,DSC,VV,139,S1B,-9.0,-13.537037,-14.0,-20.0,1.912076
942364,2019-03-29,62588991,252,"Permanent græs, normalt udbytte",2.21,DSC,VV,139,S1B,-11.0,-13.669903,-14.0,-17.0,1.535270
942365,2019-03-29,62174897,252,"Permanent græs, normalt udbytte",1.88,DSC,VV,139,S1B,-6.0,-9.358621,-9.0,-14.0,1.556636
942366,2019-03-29,62131272,252,"Permanent græs, normalt udbytte",2.54,DSC,VV,139,S1B,NaN,NaN,NaN,NaN,NaN


time: 676 ms


In [258]:
for i, polarization in enumerate(['VV', 'VH', 'VV-VH']):
    df_polarization = get_df(polygons_year=2019, 
                             satellite_dates=slice('2019-01-01', '2019-01-15'), 
                             fields='all', 
                             satellite='all', 
                             polarization=polarization,
                             netcdf_path=netcdf_path)
    
    # Pivot the df (https://stackoverflow.com/a/37790707/12045808)
    df_polarization = df_polarization.pivot(index='field_id', columns='date', values='stats_mean')
    
    # Merge the polarization dataframes into one dataframe
    df_polarization.columns = [str(col)[:10]+f'_{polarization}' for col in df_polarization.columns]  # Add polarization to column names
    if i == 0:
        df = df_polarization
    else:
        df = df.merge(df_polarization, left_index=True, right_index=True)  # The field_ids are the indices
        
# Drop fields having any date with a nan value, and pick num_fields from the remainder
df = df.dropna()

time: 901 ms


In [260]:
df.index

Int64Index([55356714, 55356721, 57365029, 57365037, 57769084, 57769086,
            57769088, 57821397, 57821414, 57821416,
            ...
            67365154, 67365158, 67365164, 67365168, 67366564, 67366737,
            67366740, 67366745, 67366754, 67366757],
           dtype='int64', name='field_id', length=55395)

time: 33.8 ms


Int64Index([55356714, 55356721, 57365029, 57365037, 57769084], dtype='int64', name='field_id')

time: 32.7 ms
